# Models(5.23)
This tutorial is to demonstrate to use simple policy gradient agents for portfolio management, specifically we deal with different model and how to visualize and train them today

## Step1: Import Packages

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys
from pathlib import Path
import os
import torch
# to make trademaster accessible from all viewpoints like ray rllib
import sys
sys.path.append('../trademaster')

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
import trademaster
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.utils import plot
from trademaster.utils import set_seed
import matplotlib.pyplot as plt
set_seed(2023)

2023-05-23 20:45:31,154	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-05-23 20:45:34,085	INFO worker.py:1454 -- Calling ray.init() again after it has already been called.
2023-05-23 20:45:34,152	INFO worker.py:1454 -- Calling ray.init() again after it has already been called.


## Take a look at Environment

In [2]:
from trademaster.environments.portfolio_management.environment import PortfolioManagementEnvironment
from ray.tune.registry import register_env
cfg = dict(
    data = dict(
        type = "PortfolioManagementDataset",
        data_path = "data/portfolio_management/sz50",
        train_path = "data/portfolio_management/sz50/train.csv",
        valid_path = "data/portfolio_management/sz50/valid.csv",
        test_path = "data/portfolio_management/sz50/test.csv",
        test_dynamic_path='data/portfolio_management/sz50/test.csv',
        tech_indicator_list = [
            "zopen",
            "zhigh",
            "zlow",
            "zadjcp",
            "zclose",
            "zd_5",
            "zd_10",
            "zd_15",
            "zd_20",
            "zd_25",
            "zd_30"
        ],
        initial_amount = 100000,
        transaction_cost_pct = 0.001
    )
)
cfg=Config(cfg)
dataset = build_dataset(cfg)
cfg2 = dict(dataset=dataset)
env = PortfolioManagementEnvironment(cfg2)
register_env("portfolio_management", lambda config: PortfolioManagementEnvironment(config))

## Play a single step with a random agent

In [3]:
from ray.rllib.algorithms.pg import PGConfig 
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
import ray
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
# ray.init(ignore_reinit_error=True)
from ray import air
from ray import tune
config = PGConfig()
# Print out some default values.
print(config.lr) 
# Update the config object.
config = config.training(lr=tune.grid_search([0.001, 0.0001]))
# Set the config object's env.
config = config.environment(env=env)
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "PG",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit()

(raylet) [2023-05-23 20:45:40,186 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 24088805376; capacity: 499963174912. Object creation will fail if spilling is required.


0.0004


(raylet) [2023-05-23 20:45:50,243 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 24034652160; capacity: 499963174912. Object creation will fail if spilling is required.
(raylet) [2023-05-23 20:46:00,311 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 23992242176; capacity: 499963174912. Object creation will fail if spilling is required.
(PG pid=69746) 2023-05-23 20:46:05,639	ERROR serialization.py:387 -- No module named 'trademaster'
(PG pid=69746) Traceback (most recent call last):
(PG pid=69746)   File "/Users/louison/opt/anaconda3/envs/TradeMaster/lib/python3.9/site-packages/ray/_private/serialization.py", line 385, in deserialize_objects
(PG pid=69746)     obj = self._deserialize_object(data, metadata, object_ref)
(PG pid=69746)   File "/Users/louison/opt/anaconda3/envs/TradeMaster/lib/python3.9/site-p

Trial name,trial_id
PG_<PortfolioManagementEnvironment instance>_be4cd_00000,be4cd_00000
PG_<PortfolioManagementEnvironment instance>_be4cd_00001,be4cd_00001


(raylet) [2023-05-23 20:46:20,466 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 23954374656; capacity: 499963174912. Object creation will fail if spilling is required.
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster'
(PG pid=69753) ModuleNotFoundError: No module named 'trademaster' [repeated 2x 

ResultGrid<[
  Result(
    error='TuneError',
    metrics={'trial_id': 'be4cd_00000'},
    path='/Users/louison/ray_results/PG/PG_<PortfolioManagementEnvironment instance>_be4cd_00000_0_lr=0.0010_2023-05-23_20-45-49',
    checkpoint=None
  ),
  Result(
    error='TuneError',
    metrics={'trial_id': 'be4cd_00001'},
    path='/Users/louison/ray_results/PG/PG_<PortfolioManagementEnvironment instance>_be4cd_00001_1_lr=0.0001_2023-05-23_20-46-05',
    checkpoint=None
  )
]>

(raylet) [2023-05-23 20:46:30,548 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 23953186816; capacity: 499963174912. Object creation will fail if spilling is required.
(raylet) [2023-05-23 20:46:40,636 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 23935815680; capacity: 499963174912. Object creation will fail if spilling is required.
(raylet) [2023-05-23 20:46:50,716 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 23935803392; capacity: 499963174912. Object creation will fail if spilling is required.
(raylet) [2023-05-23 20:47:00,796 E 69732 3621693] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-05-23_20-45-27_466441_69707 is over 95% full, available space: 23935787008; capacity: 499963174912. Object 

## Print the model
note that this can only be done with pytorch version of the model

In [5]:
pol = pg_trainer.get_policy()
print(pol)
# way1 to print the model paramters
for k, v in pol.get_weights().items():
    print(k, v.shape)
# way2 to print the model
model = pol.model
print(model)
# beside the defined model, 
# we also have flatten and various other predefined components
# which you can look for further details in 
# 'ray.rllib.models.torch.complex_input_net.ComplexInputNetwork'
print(model.flatten)
print(model.one_hot)
print(model.cnns)
# if you want to look into the source code for prototype of the model see here
print(type(model)) # of class 'ray.rllib.models.torch.complex_input_net.ComplexInputNetwork'

PGTorchPolicy
post_fc_stack._hidden_layers.0._model.0.weight (30, 256)
post_fc_stack._hidden_layers.0._model.0.bias (30,)
post_fc_stack._hidden_layers.1._model.0.weight (520, 30)
post_fc_stack._hidden_layers.1._model.0.bias (520,)
post_fc_stack._hidden_layers.2._model.0.weight (321, 520)
post_fc_stack._hidden_layers.2._model.0.bias (321,)
post_fc_stack._value_branch_separate.0._model.0.weight (30, 256)
post_fc_stack._value_branch_separate.0._model.0.bias (30,)
post_fc_stack._value_branch_separate.1._model.0.weight (520, 30)
post_fc_stack._value_branch_separate.1._model.0.bias (520,)
post_fc_stack._value_branch_separate.2._model.0.weight (321, 520)
post_fc_stack._value_branch_separate.2._model.0.bias (321,)
post_fc_stack._value_branch._model.0.weight (1, 321)
post_fc_stack._value_branch._model.0.bias (1,)
logits_layer._model.0.weight (102, 321)
logits_layer._model.0.bias (102,)
value_layer._model.0.weight (1, 321)
value_layer._model.0.bias (1,)
ComplexInputNetwork(
  (post_fc_stack): Fu

## Try and construct and pass a sample_batch into the model

In [6]:
obs = env.reset()
print(obs.shape)
sample_batch = (dict(obs = torch.Tensor([obs, obs])), None, None)
model_output = (model(*sample_batch))
print(model_output, model_output[0].shape)

(11, 50)
(tensor([[-2.5922e-04,  3.7101e-04, -2.4454e-04, -1.4447e-04,  1.9354e-04,
          2.4718e-04, -5.6721e-04,  3.5246e-04,  4.6053e-05,  6.9738e-04,
          3.2152e-04, -1.9862e-04,  1.3373e-04,  7.9480e-05,  4.3697e-04,
          2.3909e-04,  1.6504e-04,  9.2378e-04, -1.7482e-04, -6.6044e-05,
          3.6818e-04,  1.5503e-04,  1.0421e-03, -2.6726e-06,  1.6905e-04,
          5.0499e-04,  1.5690e-04,  3.1352e-04, -2.2751e-05, -4.5401e-04,
          7.2833e-05,  5.3493e-04,  5.8630e-05, -5.0553e-04, -8.5027e-05,
          1.7867e-04, -3.0604e-05,  3.9327e-04, -3.9780e-04, -1.5325e-04,
         -2.6746e-05, -2.7179e-04, -8.5194e-04,  3.7674e-04, -3.5922e-04,
          3.2475e-04,  1.0711e-04,  6.9320e-04,  4.3957e-04,  4.9434e-05,
          6.1676e-04, -1.6000e-04,  4.2720e-05,  5.0777e-04,  5.1159e-04,
         -4.6186e-04, -2.8623e-04, -3.6844e-04,  8.1008e-04,  4.6258e-05,
          1.4441e-04,  1.7621e-04, -4.9282e-04, -4.1605e-04,  7.9114e-04,
          2.8525e-05, -5.577

In [7]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models import MODEL_DEFAULTS
# model = ModelCatalog.get_model_v2(env.observation_space, env.action_space, int(env.action_space.shape[0]), model_config=MODEL_DEFAULTS)
model = ModelCatalog.get_model_v2(env.observation_space, env.action_space, int(env.action_space.shape[0]), model_config=MODEL_DEFAULTS, framework="torch")
print(model)

ComplexInputNetwork(
  (post_fc_stack): FullyConnectedNetwork(
    (_hidden_layers): Sequential()
    (_value_branch_separate): Sequential()
    (_value_branch): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=True)
      )
    )
  )
  (logits_layer): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=51, bias=True)
    )
  )
  (value_layer): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=True)
    )
  )
)


## Let's get a more complex model and try to train!

In [8]:
trainer_cfg = dict(
    rollout_fragment_length = 200,
    # explore = True,
    framework = "torch",
    model = {
        # "post_fcnet_hiddens" : [256, 256],
        # use_attention = True,
        "use_lstm": True,
        # Max seq len for training the LSTM, defaults to 20.
        "max_seq_len": 1000,
        # Size of the LSTM cell.
        "lstm_cell_size": 64,
        # Whether to feed a_{t-1} to LSTM (one-hot encoded if discrete).
        "lstm_use_prev_action": True,
        # Whether to feed r_{t-1} to LSTM.
        "lstm_use_prev_reward": False,
        # Whether the LSTM is time-major (TxBx..) or batch-major (BxTx..).
    }
    
)
ray.init(ignore_reinit_error=True)
trainer_cfg["env"] = "portfolio_management"
trainer_cfg["env_config"] = dict(dataset=dataset, task="train", device="cpu")
pg_trainer2 = PGTrainer(trainer_cfg, env="portfolio_management")
pol = pg_trainer2.get_policy()
model = pol.model
print(type(model))
print(model)
print(model.flatten)

2023-05-23 16:56:47,681	INFO worker.py:973 -- Calling ray.init() again after it has already been called.


torch


2023-05-23 16:56:48,185	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2023-05-23 16:56:48,267	WARNING util.py:65 -- Install gputil for GPU system monitoring.


<class 'ray.rllib.models.catalog.ComplexInputNetwork_as_LSTMWrapper'>
ComplexInputNetwork_as_LSTMWrapper(
  (post_fc_stack): FullyConnectedNetwork(
    (_hidden_layers): Sequential()
    (_value_branch_separate): Sequential()
    (_value_branch): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=True)
      )
    )
  )
  (lstm): LSTM(307, 64, batch_first=True)
  (_logits_branch): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=64, out_features=102, bias=True)
    )
  )
  (_value_branch): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)
{0: FullyConnectedNetwork(
  (_hidden_layers): Sequential(
    (0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=550, out_features=256, bias=True)
        (1): Tanh()
      )
    )
    (1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): Tanh()
     

In [ ]:
for e in range(3):
    pg_trainer2.train()

## Save the model

In [10]:
from trademaster.utils import get_attr, save_object, load_object

# model level saving & loading
# torch.save(model.state_dict(), "model.pkl")
# model.load_state_dict(torch.load("model.pkl"))

# trainer level saving & loading
obj = pg_trainer2.save_to_object()
save_object(obj, "pg_trainer2.pkl")
# obj2 = load_object("pg_trainer2.pkl")
pg_trainer2.restore_from_object(obj)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [5]:
import ray

print(ray.__version__)

2.4.0


PGTrainer


## How to define your personal model